<div class="alert alert-block alert-success"><font color="black"><h1><left>Scrub MIMIC ICU Data for Intestinal Conditions</left></h1></font></div>

In [7]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [8]:
path = 'E:/Education/CCSU-Thesis-2024/Data/'
file1 = path + 'pd_notes.snappy3.parquet'
file2 = path + 'discharge_notes.snappy3.parquet'

In [9]:
# How many notes from each file
pn_notes = pd.read_parquet(file1,engine='auto',dtype_backend='numpy_nullable')
print('admissions: ',len(pn_notes['hadm_id'].unique()))
discharge_notes = pd.read_parquet(file2,engine='auto',dtype_backend='numpy_nullable')
print('discharge-notes: ',len(discharge_notes['hadm_id'].unique()))
print('avg_pd_notes_per_admiss', len(pn_notes)/len(pn_notes['hadm_id'].unique()))

admissions:  5379
discharge-notes:  5359
avg_pd_notes_per_admiss 1.0


In [10]:
# This identifies that 52 admissions do not have discharge notes
import pandasql as ps

qintsl = pn_notes[['primary_icd10_code', 'hadm_id']]
qints2 = discharge_notes[['subject_id', 'hadm_id']]
# print(qintsl.dtypes)
q1 = """SELECT a.primary_icd10_code, count(distinct a.hadm_id) as totals FROM qintsl a 
        group by a.primary_icd10_code"""  
#        WHERE NOT EXISTS (SELECT *
#                  FROM qints2 b 
#                  where a.subject_id = b.subject_id 
#                  AND a.hadm_id = b.hadm_id)
#      """

p1 = ps.sqldf(q1, locals())
print(p1)
#print('Missing discharge notes count', len(p1))

   primary_icd10_code  totals
0              A41.51     151
1               A41.9     881
2               I21.4     950
3              I25.10    1216
4               I34.0      74
5               I35.2     155
6              I50.23      92
7               I50.9     301
8               J18.9     324
9               J69.0     348
10             J96.00     625
11              N17.9     262


In [11]:
# print 5 notes and their length to guarantee format before processing
print(pn_notes.columns) 
note_len = [len(x) for x in pn_notes['text']]
print(note_len[1:5])
pn_notes.iloc[[1,2,3,4,5],[2]]

Index(['subject_id', 'hadm_id', 'text', 'note_length', 'primary_icd9_code',
       'primary_icd10_code', 'icd_category', 'stay_days', 'admission_type',
       'admittime', 'dischtime', 'deathtime', 'insurance', 'language',
       'religion', 'marital_status', 'ethnicity', 'diagnosis', 'ethnic_group'],
      dtype='object')
[7912, 9896, 11662, 4894]


,text
1,"ccu npn s- having cpain left, l shoulder, arm,..."
2,"ADMISSION NOTE 87 YR OLD MAN WITH H/O CAD, CVA..."
3,CCU Progress Note: This is a 58 yr old male P...
4,Nsg Adm note Mr. [**Known lastname 185**] is a...
5,Nursing Admission Note 0700-1900. Pt transfere...


In [12]:
# note counts by admission type
pn_notes.groupby('admission_type')['hadm_id'].count()

admission_type
EMERGENCY    5126
URGENT        253
Name: hadm_id, dtype: Int64

In [13]:
def store_med_tags(df):
    
    import operator

    tags_dict = {}
    for row in df.itertuples():
        tags = findall_med_tags(row.text)
        if tags is not None:
            for tag in tags:
                temp_tag = str(tag)
#                if len(tags) > 0 :
#                    print('tag: ',tags)
                if temp_tag in tags_dict.keys():
                    tags_dict[temp_tag] += 1
                else:
                    tags_dict[temp_tag] = 1
    
    sorted_d = dict( sorted(tags_dict.items(), key=operator.itemgetter(1),reverse=True))
#    print('Dictionary in descending order by value : ',len(sorted_d))
     
    return sorted_d
#    return sorted_d


def findall_med_tags(note_data):

# https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
    #print('in data length: ', len(note_data))
    
    regex2 = r"(\[(.+?)\])"
    
#    note_data = re.sub(r'\[.*?:.*?\]', ' ', note_data)
#    note_data = re.sub(r'\[\*\*.*?\*\*\]', ' ', note_data)
#    note_data = re.sub(r'\s{2,99}', ' ', note_data)
#    note_data = re.sub(r'\n', ' ', note_data)
#    note_data = re.sub(regex, ' ', note_data)
#    note_data = note_data.upper()
    tags= re.findall(regex2, note_data)
    
#    if len(tags) > 0 :
#        print('tag length: ', len(tags))
#        print('tags:\n ', tags)
        
    return tags


def write_dict_csv(outfile, adict):

    
    import csv
    print('Write input Dictionary: ',len(adict.keys()))
    field_names = ['word','count']
    
    

    with open(outfile, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=field_names)
        writer.writeheader()
        for key in adict:
#            if adict[key] > 300:
            writer.writerow({'word': key, 'count': adict[key]})
    csvfile.close()   
    return
    
file2 = path + 'big_notes_tags_4-7-23.csv'
adict = store_med_tags(pn_notes)
if adict is not None:
    write_dict_csv(file2, adict)


Write input Dictionary:  27605


In [14]:
# Routine to remove unneccessary characters
import re

def preprocess1(x):
    
#    print('in2a: ', 'x length ', len(x),  '', x[0:20])
    y = re.sub('\\[(.*?)\\]', '', x)
#    print('in2b: ', 'y length ', len(y),  '', y[0:20])
    y = re.sub('[0-9]+\.', '', y)  # remove 1.2. since the segmenter segments based on this
#    print('in2c: ', 'y length ', len(y),  '', y[0:20])
    y = re.sub('dr\.', 'doctor', y)
#    print('in2d: ', 'y length ', len(y),  '', y[0:20])
    y = re.sub('m\.d\.', 'md', y)
#    print('in2e: ', 'y length ', len(y),  '', y[0:20])
    y = re.sub('admission date:', '', y)
#    print('in2f: ', 'y length ', len(y),  '', y[0:20])
    y = re.sub('discharge date:', '', y)
#    print('in2g: ', 'y length ', len(y),  '', y[0:20])
    y = re.sub('--|__|==', ' ', y)
#   remove punctuation   -- new change 11/16/2024
    y = re.sub(r'[^\w\s]', ' ', y)
     # remove, digits, spaces
#    print('in2h: ', 'y length ', len(y),  ' ', y[0:20])
    y = y.translate(str.maketrans("", "", string.digits))
#    print('in2i: ', 'y length ', len(y),  ' ', y[0:20])
    y = " ".join(y.split())
#    print('in2j: ', 'y length ', len(y),  ' ', y[0:20])
    
    return y

In [15]:
# function to process each note, call above function, get rid of line feeds, and make lowercase
from tqdm import tqdm, trange 
def preprocessing(df_less_n):
#    print('in2: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    df_less_n['text'] = df_less_n['text'].fillna(' ')
#    print('in3: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    df_less_n['text'] = df_less_n['text'].str.replace('\n', ' ')
#    print('in4: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    df_less_n['text'] = df_less_n['text'].str.replace('\r', ' ')
#    print('in5: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    df_less_n['text'] = df_less_n['text'].apply(str.strip)
#    print('in6: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    df_less_n['text'] = df_less_n['text'].str.lower()
#    print('in7: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    df_less_n['text'] = df_less_n['text'].apply(lambda x: preprocess1(x))
#    print('in8: ', 'df_less_n ', df_less_n.shape[0], ' ', df_less_n.iloc[0].text[0:20])
    return df_less_n



In [10]:
def create_subsequences(df_less_n):
#to get 318 words chunks for readmission tasks
    
    from tqdm import tqdm
    df_len = len(df_less_n)
    want=pd.DataFrame({'hadm_id':[],'text':[],'icd10_code':[], 'category':[]})
    
    for i in tqdm(range(df_len)):
        x=df_less_n.text.iloc[i].split()
        n=int(len(x)/318)
        for j in range(n):
            #want=want.append({'TEXT':' '.join(x[j*318:(j+1)*318]),'Label':df_less_n.OUTPUT_LABEL.iloc[i],'ID':df_less_n.HADM_ID.iloc[i]},ignore_index=True)
            temp_dict = {'text':' '.join(x[j*318:(j+1)*318]),'icd10_code':df_less_n.primary_icd10_code.iloc[i],
                         'hadm_id':df_less_n.hadm_id.iloc[i], 'category':df_less_n.icd_category.iloc[i]}
            want = pd.concat([want, pd.DataFrame(temp_dict, index=[0])], ignore_index=True)
            
        if len(x)%318>10:
            #want=want.append({'TEXT':' '.join(x[-(len(x)%318):]),'Label':df_less_n.OUTPUT_LABEL.iloc[i],'ID':df_less_n.HADM_ID.iloc[i]},ignore_index=True)
            temp_dict = {'text':' '.join(x[-(len(x)%318):]),'icd10_code':df_less_n.primary_icd10_code.iloc[i],
                         'hadm_id':df_less_n.hadm_id.iloc[i], 'category':df_less_n.icd_category.iloc[i]}
            want = pd.concat([want, pd.DataFrame(temp_dict, index=[0])], ignore_index=True)
    
    return want


In [24]:
def preprocessingb(df_less_n): 
    #print(df_less_n.columns)
    df_less_n['text']=df_less_n['text'].fillna(' ')
    df_less_n['text']=df_less_n['text'].str.replace('\n',' ')
    df_less_n['text']=df_less_n['text'].str.replace('\r',' ')
    df_less_n['text']=df_less_n['text'].apply(str.strip)
    df_less_n['text']=df_less_n['text'].str.lower()

    df_less_n['text']=df_less_n['text'].apply(lambda x: preprocess1(x))

    #to get 318 words chunks for readmission tasks
    #want = create_subsequences(df_less_n)
    #return want
    return(df_less_n[['hadm_id', 'text', 'primary_icd10_code', 'icd_category']]) 


In [25]:
def preprocessingc(df_less_n): 
    #print(df_less_n.columns)
    df_less_n['text']=df_less_n['text'].fillna(' ')
    df_less_n['text']=df_less_n['text'].str.replace('\n',' ')
    df_less_n['text']=df_less_n['text'].str.replace('\r',' ')
    df_less_n['text']=df_less_n['text'].apply(str.strip)
    df_less_n['text']=df_less_n['text'].str.lower()

    df_less_n['text']=df_less_n['text'].apply(lambda x: preprocess1(x))

    #to get 318 words chunks for readmission tasks
    #want = create_subsequences(df_less_n)
    #return want
    return(df_less_n[['hadm_id', 'text', 'primary_icd10_code', 'icd_category']]) 


In [26]:
# run preprocess for discharge and adm notes
import string
fdischarge_notes = preprocessingb(discharge_notes)
fdischarge_notes.columns =  ['hadm_id', 'text', 'icd10_code', 'category']
fpn_notes = preprocessingb(pn_notes)
fpn_notes.columns =  ['hadm_id', 'text', 'icd10_code', 'category']
# print first 2 texts columns in each dataset as a sanity check after cleanup
full_fpn_notes = preprocessingc(pn_notes)
full_fpn_notes.columns =  ['hadm_id', 'text', 'icd10_code', 'category']

print(len(fpn_notes), len(fdischarge_notes), len(full_fpn_notes))
print(fpn_notes[0:2])
print(fdischarge_notes[0:2])

5379 5359 5379
   hadm_id                                               text icd10_code  \
0   145834  micu progress nursing note patient arrived in ...      A41.9   
1   143045  ccu npn s having cpain left l shoulder arm bac...     I25.10   

  category  
0    Blood  
1    Heart  
   hadm_id                                               text icd10_code  \
0   145834  date of birth sex m service medicine chief com...      A41.9   
1   143045  date of birth sex f service cardiac surgery ch...     I25.10   

  category  
0    Blood  
1    Heart  


In [27]:
print(len(fpn_notes), len(fdischarge_notes), len(full_fpn_notes))
print(fpn_notes[0:2].text[0:10])
print(fdischarge_notes[0:2].text[0:10])

5379 5359 5379
0    micu progress nursing note patient arrived in ...
1    ccu npn s having cpain left l shoulder arm bac...
Name: text, dtype: object
0    date of birth sex m service medicine chief com...
1    date of birth sex f service cardiac surgery ch...
Name: text, dtype: object


In [28]:
path = 'E:/Education/CCSU-Thesis-2024/Data/'
f1name = 'pd_notes_cleanb.snappy3.parquet'
f3name = 'full_pd_notes_cleanb.snappy3.parquet'
f2name = 'discharge_notes_cleanb.snappy3.parquet'
#fpn_notes.to_parquet(path + f1name,compression='snappy',index=None)  
full_fpn_notes.to_parquet(path + f3name,compression='snappy',index=None)  
#fdischarge_notes.to_parquet(path + f2name,compression='snappy',index=None)  

In [29]:
print(fpn_notes.groupby('icd10_code')['hadm_id'].nunique())
print(full_fpn_notes.groupby('icd10_code')['hadm_id'].nunique())

icd10_code
A41.51     151
A41.9      881
I21.4      950
I25.10    1216
I34.0       74
I35.2      155
I50.23      92
I50.9      301
J18.9      324
J69.0      348
J96.00     625
N17.9      262
Name: hadm_id, dtype: int64
icd10_code
A41.51     151
A41.9      881
I21.4      950
I25.10    1216
I34.0       74
I35.2      155
I50.23      92
I50.9      301
J18.9      324
J69.0      348
J96.00     625
N17.9      262
Name: hadm_id, dtype: int64


In [44]:
import boto3, os
s3 = boto3.resource('s3')


In [45]:
subdir = 'Data/' 
S3_path = subdir + f1name
out_file = path + f1name
s3.meta.client.upload_file(out_file, 'depratti-masters-thesis', S3_path)
S3_path = subdir + f2name
out_file = path + f2name
s3.meta.client.upload_file(out_file, 'depratti-masters-thesis', S3_path)

In [30]:
print('fpn_notes shape: ', fpn_notes.shape)
print('fpn_notes columns: ', fpn_notes.columns)
print('full fpn_notes shape: ', full_fpn_notes.shape)
print('full_fpn_notes columns: ', full_fpn_notes.columns)

fpn_notes shape:  (5379, 4)
fpn_notes columns:  Index(['hadm_id', 'text', 'icd10_code', 'category'], dtype='object')
full fpn_notes shape:  (5379, 4)
full_fpn_notes columns:  Index(['hadm_id', 'text', 'icd10_code', 'category'], dtype='object')


In [31]:
print(fpn_notes.groupby('category')['hadm_id'].count())
fpn_notes.groupby('icd10_code')['hadm_id'].count()[1:20]

category
Blood     1032
Heart     2788
Kidney     262
Lungs     1297
Name: hadm_id, dtype: Int64


icd10_code
A41.9      881
I21.4      950
I25.10    1216
I34.0       74
I35.2      155
I50.23      92
I50.9      301
J18.9      324
J69.0      348
J96.00     625
N17.9      262
Name: hadm_id, dtype: Int64

In [32]:
print(fpn_notes.groupby('icd10_code')['hadm_id'].nunique())

icd10_code
A41.51     151
A41.9      881
I21.4      950
I25.10    1216
I34.0       74
I35.2      155
I50.23      92
I50.9      301
J18.9      324
J69.0      348
J96.00     625
N17.9      262
Name: hadm_id, dtype: int64


In [33]:
print(len(fdischarge_notes), len(fdischarge_notes.hadm_id.unique()), 
      len(fpn_notes), len(fpn_notes.hadm_id.unique()))
print(fpn_notes.groupby('category')['hadm_id'].nunique())
full_fpn_notes.groupby('category')['hadm_id'].nunique()

5359 5359 5379 5379
category
Blood     1032
Heart     2788
Kidney     262
Lungs     1297
Name: hadm_id, dtype: int64


category
Blood     1032
Heart     2788
Kidney     262
Lungs     1297
Name: hadm_id, dtype: int64